<a href="https://colab.research.google.com/github/nathanbollig/distributed-mutation/blob/main/explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore Synthetic Data and Classifier

In Phase 1 and 2 of this project, synthetic viral sequence data is generated and a classification model is trained on 80% of this data. The result of running the `phases_1_and_2.py` script is a set results objects saved to disk into 6 pickle files. After performing Phase 1 and Phase 2, this notebook loads in these files and explores their contents.

In [1]:
import numpy as np
from tensorflow import keras
import pickle

## Phase 1 and 2

This section follows the instructions in the project README to generate data and train a classifier.

In [2]:
# Clone the git repository
!git clone https://github.com/nathanbollig/distributed-mutation

Cloning into 'distributed-mutation'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 46 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [3]:
# Apply requirements.txt
%cd distributed-mutation
!pip install -r requirements.txt

/content/distributed-mutation
     |████████████████████████████████| 374 kB 5.3 MB/s 
     |████████████████████████████████| 312 kB 57.9 MB/s 
     |████████████████████████████████| 7.1 MB 15.1 MB/s 
     |████████████████████████████████| 110.5 MB 1.3 kB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 2.9 MB 46.3 MB/s 
     |████████████████████████████████| 3.8 MB 45.2 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
     |████████████████████████████████| 503 kB 56.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=dbe05188e60adbd7c205957d4e083af7ec9420f1c85abf0bb2bde563d40b2d60
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall:

The above conflicts are acceptable because these conflicts are with pre-installed packages in this environment that I don't believe are needed for executing my code. May now need to restart the runtime (and therefore change directory again).

In [ ]:
# Run script
%cd distributed-mutation
!bash phases_1_and_2.sh

/content/distributed-mutation
Using TensorFlow backend.
tcmalloc: large alloc 4800004096 bytes == 0x55994a792000 @  0x7f3ac9792001 0x7f3ac73367b5 0x7f3ac739ac00 0x7f3ac739ca9f 0x7f3ac7433078 0x5599263c7544 0x5599263c7240 0x55992643b627 0x5599264359ee 0x5599263c8bda 0x559926436915 0x5599263c8afa 0x559926436915 0x5599264359ee 0x5599263c8bda 0x559926437737 0x5599264359ee 0x5599264356f3 0x5599264ff4c2 0x5599264ff83d 0x5599264ff6e6 0x5599264d7163 0x5599264d6e0c 0x7f3ac857abf7 0x5599264d6cea
tcmalloc: large alloc 3840000000 bytes == 0x559a68936000 @  0x7f3ac97901e7 0x7f3ac7336631 0x7f3ac739acc8 0x7f3ac739ade3 0x7f3ac7425f06 0x7f3ac7426368 0x5599264af409 0x559926436e7a 0x5599264359ee 0x5599263c8bda 0x559926437737 0x5599264359ee 0x5599263c8bda 0x559926436915 0x5599264b9cf8 0x5599264afcae 0x55992649fae5 0x5599263d6224 0x5599264070a4 0x5599263c7c52 0x55992643ac25 0x559926435ced 0x5599263c8bda 0x559926437737 0x5599264359ee 0x5599263c8bda 0x559926437737 0x5599264359ee 0x5599264356f3 0x5599264ff4c2

## Explore Phase 1 and 2 Output

### Read in results

In [12]:
# Load model
model = keras.models.load_model('model.tf')

In [13]:
# Load pickled objects
with open("aa_vocab.pkl", 'rb') as pfile:
    aa_vocab = pickle.load(pfile)
with open("generator.pkl", 'rb') as pfile:
    gen = pickle.load(pfile)
with open("result.pkl", 'rb') as pfile:
    result = pickle.load(pfile)

In [14]:
def get_test_data():
    # Load test data
    with open('data_test.txt') as f:
        lines = f.readlines()

    '''
    Convert string representation of sequence in `sequences` to integer-encoded list
    '''
    def seq_str_to_list(s):
        seq = s.split(',')
        return list(map(int, seq))

    # Read file into lists
    X = []
    y = []
    for line in lines:
        split_line = line.split('\t')
        seq = seq_str_to_list(split_line[0])
        label = int(split_line[1])
        X.append(seq)
        y.append(label)

    # Convert lists to arrays
    X = np.array(X, dtype=int)
    y = np.array(y, dtype=int)

    return X, y

X, y = get_test_data()

In [15]:
X[5]

array([ 1, 15,  7,  1, 19,  8,  6,  7,  0,  7,  1,  1, 14, 11,  1, 19,  9,
       11,  6,  5,  1,  9,  9,  9, 15, 18, 11,  2, 12, 15,  5,  3, 10, 10,
       13, 19, 10,  8, 10, 10, 18,  0, 16,  9, 14, 19, 11, 10,  4, 19, 14,
       16, 12, 13, 11, 12, 11,  2,  8,  2])

In [16]:
y[5]

1

### Model and validation results

The Keras model object is stored in `model`.

In [17]:
type(model)

tensorflow.python.keras.engine.sequential.Sequential

During the training of this model, a dictionary was created of training set and validation set performance. We can display the values in this dictionary to see the accuracy of the stored model on its training and validation set.

In [ ]:
result

{'model_train_accuracy': 0.979525, 'model_val_accuracy': 0.97968}

### Synthetic sequence data

The data are sequences (variable names prefixed with `X`) and their labels (prefixed with `y`). Data is split into training, validation, and test sets. The training set was used to train the model and the validation set was used to measure the performance reported in the `result` dictionary. The test set has not yet been used for model training or evaluation.

The data should be 80% training, 10% validation, and 10% test, with the total number of sequences as specified in the `phases_1_and_2.sh` script.

In [ ]:
len(X_train), len(X_val), len(X_test)

(800000, 100000, 100000)

As expected, there are the same number of labels.

In [ ]:
len(y_train), len(y_val), len(y_test)

(800000, 100000, 100000)

The sequence variables (`X_`*) are numpy arrays, where each sequence is represented by a 60 x 20 matrix.

In [ ]:
X_train.shape

(800000, 60, 20)

Each of the 60 positions in the sequence is represented by a one-hot vector of length 20. We assume that 20 is the size of the character alphabet. For example, a single sequence looks like the following matrix.

In [ ]:
X_train[42]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

The binary sequence label is an integer. The value 1 represents positive, and 0 represents negative. For example, the label for the above sequence is shown below.

In [ ]:
y_train[42]

1

### Applying the model to a sequence

We can apply the stored model to a sequence to get a prediction, using the TensorFlow model objects's API. For example, suppose we want to apply the model to sequences at index 42 and 43 in the training set.

In [ ]:
model.predict(X_train[42:44])

array([[0.00996366],
       [0.01680388]], dtype=float32)

We see the prediction for each sequence as a number between 0 and 1. In this case, they are both close to 1, indicating higher confidence of positive. These predictions are correct in this context, as shown below.

In [ ]:
y_train[42:44]

array([1, 0])

### Amino acid vocabulary

The pickle file also included the amino acid vocabulary used for the encoding.

In [ ]:
print(aa_vocab)

['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']


In [ ]:
len(aa_vocab)

20

This associates an index in the range 0-19 (as described above in relation to the one-hot representation of sequences) to a specific character that reflects an amino acid in the biological sequence.

### Markov model (HMMGenerator object)

Finally, the pickle file includes the `HMMGenerator` object used to synthesize the data.

In [ ]:
type(gen)

HMM_generator_motif.HMMGenerator

This object has fields that determine the structure behind the synthesized data. For example, the sequence lengths, where the active site starts in a sequence, how long the active site is, the class proportion, the intensity of the positive class signal (as described in my report), emission probability distributions, transition mutation probabilities, and some others.

In [ ]:
print(gen.__dict__.keys())

dict_keys(['seq_length', 'start', 'active_site_length', 'p', 'class_signal', 'aa_list', 'background_emission', 'state0_emission', 'state1_emissions', 'transmat', 'startprob', 'emissionprob', 'n_components', 'model'])


The `HMMGenerator` class is a wrapper around a Multinomial Hidden Markov Model implemented in `hmmlearn`. The field `gen.model` contains the `hmmlearn` model used to synthesize data.

In [ ]:
type(gen.model)

hmmlearn.hmm.MultinomialHMM

## Using the generator to classify novel sequences

The `HMMGenerator` class is capable of predicting the class 1 probability of a sequence under the existing HMM. Suppose we take a test item.

In [ ]:
x = X_test[42]

In [ ]:
y = y_test[42]
y

1

This is a negative instance. Make a mutation at position 35.

In [ ]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
x[25]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [ ]:
old_char_idx = np.argmax(x[25])
x[25][old_char_idx]

1.0

In [ ]:
x[25][old_char_idx] = 0.0

In [ ]:
x[25][11] = 1.0

In [ ]:
aa_vocab[16]

'T'

In [ ]:
aa_vocab[11]

'K'

This corresponds to substituting the 16th character in the alphabet ('T') with the 11th character ('K'). Now we can predict the class label using the model (after reshaping into a batch of size 1), and predict the class label under the generator model (after converted to a sequence of indices rather than one-hot encoding).

In [ ]:
# Model prediction on mutation
model.predict(x.reshape(1,60,20))

array([[0.9999988]], dtype=float32)

In [ ]:
# Generator posterior prob of positive class
gen.predict_proba(np.argmax(x, axis=1))

0.9999979333135554

Make a few more mutations that I know should make it look like a positive sequence, then see that the generator indicates this looks more like a positive sequence, and the model correctly predicts this as well.

In [ ]:
import numpy as np

# Set the following characters starting at index 26: 'RSFIED'
chars = 'RSFIED'

for i in range(26, 32):
    # Get the new char index
    char = chars[i-26]
    new_char_idx = aa_vocab.index(char)

    # Reset current char to 0
    x[i][np.argmax(x[i])] = 0.0

    # Make substitution for new char
    x[i][new_char_idx] = 1.0

In [ ]:
# Generator posterior prob of positive class
gen.predict_proba(np.argmax(x, axis=1))

0.9999999978548146

In [ ]:
# Model prediction on mutation
model.predict(x.reshape(1,60,20))

array([[1.]], dtype=float32)

### Copy pickle files to Google Drive

First need to mount Google Drive for this to work, and have a folder there called '744'.

In [ ]:
# Copy files to Drive

!cp *.* ../drive/MyDrive/744/